<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/cmv_modes_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Jeevesh8/arg_mining

Cloning into 'arg_mining'...
remote: Enumerating objects: 1006, done.
remote: Counting objects: 100% (1006/1006), done.
remote: Compressing objects: 100% (663/663), done.
remote: Total 1006 (delta 616), reused 618 (delta 296), pack-reused 0
Receiving objects: 100% (1006/1006), 18.65 MiB | 3.91 MiB/s, done.
Resolving deltas: 100% (616/616), done.
Checking out files: 100% (188/188), done.


In [ ]:
!pip install transformers

In [ ]:
!pip install flax

In [ ]:
from arg_mining.datasets.cmv_modes import load_dataset, data_config

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased',
                                          bos_token="[CLS]",
                                          eos_token="[SEP]")
tokenizer.add_tokens(data_config["special_tokens"])

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

15

In [ ]:
from transformers import LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096',)
tokenizer.add_tokens(data_config["special_tokens"])

15

In [ ]:
with open('./arg_mining/Discourse_Markers.txt') as f:
    discourse_markers = [dm.strip() for dm in f.readlines()]

In [ ]:
train_dataset, _, _ = load_dataset(tokenizer=tokenizer,
                       train_sz=100,
                       test_sz=0, 
                       #mask_tokens=discourse_markers,
                       )

In [ ]:
comp_nos = {"B-C":0,
            "I-C":0,
            "B-P":0,
            "I-P":0,
            "O":0}

In [ ]:
relation_nos = {k: 0 for k in data_config["adv_relations_map"].keys()}

In [ ]:
num_to_key = {i:k for i, k in enumerate(data_config["adv_relations_map"].keys())}

In [ ]:
ac_dict = data_config["arg_components"]

In [ ]:
import numpy as np

In [ ]:
for tokenized_threads, masked_threads, comp_type_labels, ref_and_rel_types in train_dataset:
    for ref_and_rel_type in np.squeeze(ref_and_rel_types):
        for (fr, to, rel_type) in ref_and_rel_type:
            if fr==0:
                break
            else:
                relation_nos[num_to_key[rel_type]] += 1
            
    tokenized_threads = np.squeeze(tokenized_threads)
    masked_threads = np.squeeze(tokenized_threads)
    comp_type_labels = np.squeeze(comp_type_labels)
    for tokenized_thread, masked_thread, comp_type_label in zip(tokenized_threads, masked_threads, comp_type_labels):
        for token_id, elem in zip(tokenized_thread, comp_type_label):
            if token_id==tokenizer.pad_token_id:
                break
            if elem==ac_dict["B-C"]:
                comp_nos["B-C"]+=1
            elif elem==ac_dict["B-P"]:
                comp_nos["B-P"]+=1
            elif elem==ac_dict["I-C"]:
                comp_nos["I-C"]+=1
            elif elem==ac_dict["I-P"]:
                comp_nos["I-P"]+=1
            elif elem==ac_dict["O"]:
                comp_nos["O"]+=1
            else:
                y = 0/0

In [ ]:
#Longformer Tokenizer, Without Discourse Markers
comp_nos, relation_nos

({'B-C': 1650, 'B-P': 1980, 'I-C': 26529, 'I-P': 36552, 'O': 28186},
 {'None': 519,
  'agreement': 421,
  'direct_attack': 283,
  'partial': 215,
  'support': 1859,
  'undercutter_attack': 330})